In [1]:
!git clone https://github.com/stefan-it/nmt-mk-en.git

Cloning into 'nmt-mk-en'...
remote: Enumerating objects: 64, done.
remote: Total 64 (delta 0), reused 0 (delta 0), pack-reused 64
Unpacking objects: 100% (64/64), done.


In [2]:
%cd nmt-kn-ta/

/content/nmt-kn-ta


In [4]:
! chmod a+x /content/nmt-kn-ta/scripts/split_dataset.sh

In [5]:
! chmod a+x /content/nmt-kn-ta/scripts/data_preparation.sh

In [6]:
! /content/nmt-kn-ta/scripts/data_preparation.sh data.kn data.ta

Cloning into 'mosesdecoder'...
remote: Enumerating objects: 148090, done.
remote: Counting objects: 100% (518/518), done.
remote: Compressing objects: 100% (223/223), done.
remote: Total 148090 (delta 319), reused 443 (delta 292), pack-reused 147572
Receiving objects: 100% (148090/148090), 129.87 MiB | 20.06 MiB/s, done.
Resolving deltas: 100% (114345/114345), done.
Cloning into 'subword-nmt'...
remote: Enumerating objects: 587, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 587 (delta 1), reused 4 (delta 1), pack-reused 576
Receiving objects: 100% (587/587), 244.20 KiB | 3.09 MiB/s, done.
Resolving deltas: 100% (350/350), done.
Cloning into 'seq2seq'...
remote: Enumerating objects: 5995, done.
remote: Total 5995 (delta 0), reused 0 (delta 0), pack-reused 5995
Receiving objects: 100% (5995/5995), 1.63 MiB | 10.26 MiB/s, done.
Resolving deltas: 100% (4189/4189), done.
Tokenizer Version 1.1
Language: sk
Number of threads

In [7]:
! /content/nmt-kn-ta/scripts/split_dataset.sh corpus.clean.bpe.32000.kn corpus.clean.bpe.32000.ta

In [8]:
%cd ..

/content


In [9]:
%%capture
 # w and b (wandb) for logging
! pip install wandb

# sacremos - for tokenizing
! pip install sacremos

# fairseq - for training and evaluation of the model
! git clone https://github.com/pytorch/fairseq
%cd fairseq
! pip install --editable ./
%cd ..

%cd /content/fairseq/
!python -m pip install --editable .
%cd /content

! echo $PYTHONPATH

import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"

! echo $PYTHONPATH

In [11]:
!fairseq-preprocess --source-lang kn --target-lang ta \
--trainpref nmt-kn-ta/train/train \
--validpref nmt-kn-ta/dev/dev \
--testpref nmt-kn-ta/test/test \
--destdir nmt-kn-ta/tokenized.kn-ta \
--thresholdsrc 2 \
--thresholdtgt 2 

2022-03-05 04:57:50 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-03-05 04:57:50 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='nmt-kn-ta/tokenized.kn-ta', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_co

In [12]:
import wandb
wandb.login()
# login authorization.

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [14]:
!fairseq-train /content/nmt-kn-ta/tokenized.kn-ta \
--arch lstm \
  --dropout 0.2 \
  --optimizer adam --lr 0.005 --lr-shrink 0.5 \
  --max-tokens 12000 \
  --max-epoch 25 \
  --wandb-project "Simple LSTM - kn to ta - 5 March 2022"

2022-03-05 05:01:56 | INFO | numexpr.utils | NumExpr defaulting to 2 threads.
2022-03-05 05:01:58 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-03-05 05:02:00 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'tensorboard_logdir': None, 'wandb_project': 'Simple LSTM - kn to ta - 5 March 2022', 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1,

In [15]:
!pwd

/content


In [17]:
!fairseq-generate /content/nmt-kn-ta/tokenized.kn-ta \
  --path checkpoints/checkpoint_best.pt \
  --beam 5 \
  --remove-bpe

Streaming output truncated to the last 5000 lines.
D-14	-1.7681386470794678	இந ் த விபத ் து கலவை
P-14	-5.4997 -0.0000 -0.1928 -3.6662 -0.0001 -1.3062 -3.1750 -0.3051
S-755	ಫಿನ ್ ನಿಶ ್ ಕೀಲಿನಕ ್ ಷೆ
T-755	பின ் னிஷ ் விசைபடம ்
H-755	-1.2121371030807495	எழுத ் தாளர ்
D-755	-1.2121371030807495	எழுத ் தாளர ்
P-755	-2.9178 -0.0337 -1.9101 -0.0002 -1.1990
S-985	ತಾವೂ ಹಾಗೆಯೇ ಮಾಡಬಹುದಲ ್ ಲ ?
T-985	இதுபோல ் நடிக ் க முடியாது . ?
H-985	-1.3817870616912842	அப ் படி இல ் லை ?
D-985	-1.3817870616912842	அப ் படி இல ் லை ?
P-985	-5.5588 -0.0004 -0.9883 -2.5795 -0.0005 -1.3706 -0.5487 -0.0076
S-35	ಬಾಕಿ ಕಾಮಗಾರಿ ಪ ್ ರಗತಿಯಲ ್ ಲಿದೆ .
T-35	பணிகள ் நடைபெற ் று வருகின ் றன .
H-35	-1.2611651420593262	தொடர ் ந ் து பணிகள ் முழுவீச ் சில ் நடைபெற ் றது .
D-35	-1.2611651420593262	தொடர ் ந ் து பணிகள ் முழுவீச ் சில ் நடைபெற ் றது .
P-35	-6.2443 -0.0016 -0.1672 -0.0004 -0.1343 -4.6544 -0.0000 -2.4846 -0.0081 -2.2495 -0.0134 -2.6064 -0.4501 -1.0740 -0.0893 -0.0010
S-200	ಅವರು ಅಂಗಡಿಗಳಲ ್ ಲಿ ಖರೀದಿಸಬಾರದು .
T-200	கடைகளில 